<a id=contents></a>

# Cleaning notebook
## Subtitle


[1. Data Inspection](#insp)

[2. Using Spark and Hadoop](#numerical)

[3. Cleaning categorical data](#categ)

[4. Cleaning text data](#text)

In [1]:
%load_ext autoreload
%autoreload 2 

import pandas as pd
import numpy as n
from pathlib import Path

import functions.functions as fn

/Users/ipreoteasa/opt/anaconda3/envs/dev/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


<a id=insp ><a/> 

## 1. Data Inspection
    
[LINK to table of contents](#contents)

In [2]:
# 4-5 min to run
raw_path = Path('data/raw')
clean, cvt = fn.clean_tweet_text(raw_path)

In [3]:
clean.head()

,Unnamed: 0,tweet_id,datetime,display_name,tweet_text,User_id,#likes,#retweets,#responses,language,...,woman,work,working,world,would,yeah,year,yes,yet,youre
0,0,1579879154463690752,2022-10-11 16:58:37+00:00,ElonMusk,"@NorthmanTrader No, it is not. I have spoken t...",44196397,16079,1608,1308,en,...,0,0,0,0,0,0,0,0,0,0
1,1,1579627776231247874,2022-10-11 00:19:44+00:00,ElonMusk,@kanyewest Talked to ye today & expressed my c...,44196397,46226,2101,3251,en,...,0,0,0,0,0,0,0,0,0,0
2,2,1579600228826591233,2022-10-10 22:30:16+00:00,ElonMusk,@Teslarati @JohnnaCrider1 It was built & paid ...,44196397,11356,506,862,en,...,0,1,0,0,0,0,0,0,0,0
3,3,1579299680491823104,2022-10-10 02:36:00+00:00,ElonMusk,@CathieDWood 💯,44196397,11930,568,1053,und,...,0,0,0,0,0,0,0,0,0,0
4,4,1579263949945835521,2022-10-10 00:14:01+00:00,ElonMusk,@WholeMarsBlog It will,44196397,3179,152,343,en,...,0,0,0,0,0,0,0,0,0,0


In [4]:
clean.to_csv('data/clean/clean_data.csv')

In [5]:
# now selecting those parts of the data that will be used only classification, i.e. the vectorized columns
cols  =list(clean.columns)
start_cvt_feats = cols.index('clean_tweet_text')+1
cvt_cols  = clean.columns[start_cvt_feats:]
cvt_cols= cvt_cols.insert(0, 'tweet_id')
feats_df = clean[cvt_cols]
feats_df.to_csv('data/clean/features/count_vect_feats.csv')

In [6]:
clean_display = clean[cols[1:start_cvt_feats]]
clean_display.to_csv('data/clean/clean_display_data.csv')

In [7]:
clean.head(1)

,Unnamed: 0,tweet_id,datetime,display_name,tweet_text,User_id,#likes,#retweets,#responses,language,...,woman,work,working,world,would,yeah,year,yes,yet,youre
0,0,1579879154463690752,2022-10-11 16:58:37+00:00,ElonMusk,"@NorthmanTrader No, it is not. I have spoken t...",44196397,16079,1608,1308,en,...,0,0,0,0,0,0,0,0,0,0


In [8]:
import functions.nlp_eda as nlp

In [9]:
# we're also gonna get the tf-idf data since that gives us a different way 
# to analytically assess the text and words

tfdf = nlp.get_tfidf_df(clean_display, 'clean_tweet_text' )

/Users/ipreoteasa/Desktop/Io/blog/IoanaFio/content/project/twitter_sentiment_tracking/functions/nlp_eda.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset=[id_col], inplace=True)


In [10]:
tfdf.head(2)

,datetime,display_name,tweet_text,User_id,#likes,#retweets,#responses,language,extracted_twitter_handles,extracted_URLs,...,work,working,world,would,yeah,year,years,yes,yet,youre
tweet_id,,,,,,,,,,,,,,,,,,,,,
1579879154463690752,2022-10-11 16:58:37+00:00,ElonMusk,"@NorthmanTrader No, it is not. I have spoken t...",44196397,16079,1608,1308,en,[@NorthmanTrader],[],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1579627776231247874,2022-10-11 00:19:44+00:00,ElonMusk,@kanyewest Talked to ye today & expressed my c...,44196397,46226,2101,3251,en,[@kanyewest],[],...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
tfdf.iloc[:, start_cvt_feats-1:].to_csv('data/clean/features/tf_idf_df.csv')

<a id=numerical ><a/> 

## 2. Using Spark and Hadoop
    
[LINK to table of contents](#contents)

After playing around and exploring the dataset i'd gathered, I realised that all my data processing was taking really long and needed to be streamlined. So I decided to switch over to Spark and Hadoop's HDFS. 

In [11]:
from pathlib import Path
import pandas as pd
import numpy as np

In [12]:
import findspark 


In [13]:
import pyspark
pyspark.find_spark_home

<module 'pyspark.find_spark_home' from '/Users/ipreoteasa/opt/anaconda3/envs/dev/lib/python3.10/site-packages/pyspark/find_spark_home.py'>

In [14]:
findspark.init('/Users/ipreoteasa/opt/anaconda3/envs/dev/lib/python3.10/site-packages/pyspark/') 

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('twitter_sentiment_tracker').master('local[*]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/12 09:38:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
df = pd.read_csv('data/clean/clean_display_data.csv',  lineterminator='\n')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507293 entries, 0 to 507292
Data columns (total 14 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Unnamed: 0                 507293 non-null  int64 
 1   tweet_id                   507293 non-null  int64 
 2   datetime                   507293 non-null  object
 3   display_name               507293 non-null  object
 4   tweet_text                 507293 non-null  object
 5   User_id                    507293 non-null  int64 
 6   #likes                     507293 non-null  int64 
 7   #retweets                  507293 non-null  int64 
 8   #responses                 507293 non-null  int64 
 9   language                   507293 non-null  object
 10  extracted_twitter_handles  507293 non-null  object
 11  extracted_URLs             507293 non-null  object
 12  extracted_hashtags         507293 non-null  object
 13  clean_tweet_text           487313 non-null  

In [21]:
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')

# now we want a very simple before and after ElonMusk's tweet proposing elections in Ukraine
cutoff_date = pd.to_datetime('2022-10-03 16:15:43+00:00',)


In [24]:

# now adding a filter column: stating whether a date is before or after this tweet. 
df['Before_or_after_controversy'] = df['datetime'].apply(lambda x : fn.is_it_before_or_after(x, cutoff_date))

In [25]:
df = df.loc[~df.datetime.isna()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 507293 entries, 0 to 507292
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype              
---  ------                       --------------   -----              
 0   Unnamed: 0                   507293 non-null  int64              
 1   tweet_id                     507293 non-null  int64              
 2   datetime                     507293 non-null  datetime64[ns, UTC]
 3   display_name                 507293 non-null  object             
 4   tweet_text                   507293 non-null  object             
 5   User_id                      507293 non-null  int64              
 6   #likes                       507293 non-null  int64              
 7   #retweets                    507293 non-null  int64              
 8   #responses                   507293 non-null  int64              
 9   language                     507293 non-null  object             
 10  extracted_twitter_handles    507

In [30]:
df.loc[df.display_name=='ElonMusk'].Before_or_after_controversy.value_counts()


After     116
Before     34
Name: Before_or_after_controversy, dtype: int64

In [27]:
df.Before_or_after_controversy.value_counts()
df.to_csv('data/clean/clean_display_data.csv')

<a id=categ ><a/> 

## 3. Cleaning categorical data
   
[LINK to table of contents](#contents)

<a id=text ><a/> 

## 4. Cleaning text data
    
[LINK to table of contents](#contents)